Lots of imports do not know what is needed and what is not needed.

if there is time it is possible to filter

In [14]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 57.6 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-ba

In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [4]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

Help function to remove stop words and lowers ------ I dont know english

In [11]:
def tokenizer_doc_len(text,all_stopwords):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  return len(tokens)

Saves the dictionary to a file called "id_length_dict.pickle"

The file must be created before running

In [24]:
parquetFile_pd = pd.read_csv("small_corpus.csv")
parquetFile = spark.createDataFrame(parquetFile_pd)
doc_title_pairs = parquetFile.select("text","id")

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)


id_doclen_dict = {}
for i in doc_title_pairs.collect():
  #print("i.id :",i.id,"len(i.text) :",len(i.text),"after :" ,tokenizer_doc_len(i.text,all_stopwords))
  id_doclen_dict[i.id] = tokenizer_doc_len(i.text,all_stopwords)

with open('id_doclen_dict.pickle', 'wb') as f:
  pickle.dump(id_doclen_dict, f)


Example of reading the file

The dictionary will be in format id:length

In [22]:
with open('id_doclen_dict.pickle', 'rb') as f:
  did2len = pickle.loads(f.read())

print(did2len)

{4055486: 694, 4080953: 3025, 4093674: 195, 4105482: 59, 4108839: 1856, 4141563: 2103, 4146044: 147, 4148655: 1498, 4169317: 311, 4189740: 3315, 4193788: 451, 4201044: 656, 4206029: 1378, 4225907: 288, 4242777: 352, 4250574: 260, 4275382: 202, 4301719: 260, 4338696: 2536, 4344526: 136, 4355398: 1880, 4361873: 642, 4390795: 621, 4427231: 380, 4432842: 1034, 4451883: 1160, 4456418: 195, 4463369: 289, 4465875: 204, 4465969: 980, 4468939: 186, 4481567: 685, 4512778: 1103, 4523354: 273, 4539416: 143, 4552880: 1273, 4553108: 695, 4567010: 722, 4627535: 205, 4649105: 246, 4659608: 97, 4660251: 127, 4668823: 509, 4673201: 46, 4676317: 100, 4694434: 454, 4730170: 1250, 4749106: 513, 4750398: 1431, 4765530: 16926, 4819795: 1404, 4840292: 49, 4875689: 315, 4876011: 1071, 4881262: 621, 4920126: 388, 4954263: 1273, 4975851: 836, 4979006: 319, 4987149: 3538, 4992439: 724, 4992612: 845, 5003908: 168, 5027882: 2829, 5028853: 2517, 5035602: 675, 5038510: 384, 5046302: 1019, 5048642: 1417, 5052869: 785,